In [ ]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 14.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=8f0190dc3fec3691ab5db64de1f18708e0b07c4eeaefd31b91aa0d4adcffd730
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [ ]:
import pandas as pd
import numpy as np
# Load functionality to manipulate dataframes
from pyspark.sql import functions as fn
import matplotlib.pyplot as plt
from pyspark.sql.functions import stddev, mean, col
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
# Functionality for computing features
from pyspark.ml import feature, regression, classification, Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import feature, regression, classification, Pipeline
from pyspark.ml.feature import Tokenizer, VectorAssembler, HashingTF, Word2Vec, StringIndexer, OneHotEncoder
from pyspark.ml import clustering
from itertools import chain
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml import classification
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, DecisionTreeClassifier
from pyspark.ml import evaluation
from pyspark.ml.evaluation import BinaryClassificationEvaluator,MulticlassClassificationEvaluator
from pyspark import keyword_only
from pyspark.ml import Transformer
from pyspark.ml.param.shared import HasInputCol, HasOutputCol, Param
#Classification Report
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
MAX_MEMORY = "45g"

spark = SparkSession \
    .builder \
    .appName("how to read csv file") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .getOrCreate()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# load master dataset
dfmaster = spark.read.format("csv").load("/content/drive/MyDrive/Data/master.csv", delimiter = ",", header = True)

#Data Preparation

In [ ]:
# create a 0/1 column for acquistions
dfmaster = dfmaster.\
  withColumn("labelacq", fn.when(col("status") == "acquired","1").otherwise("0"))

In [ ]:
# number of rows in master table
print(dfmaster.count())

49445


In [ ]:
dfmaster

DataFrame[_c0: string, permalink: string, name: string, market: string, funding_total_usd: string, status: string, country_code: string, city: string, funding_rounds: string, founded_year: string, quarter_new: string, age: string, permalink_agg: string, count_investor: string, time_to_first_funding: string, investor_country_codes: string, funding_round_types: string, total_raised_usd: string, permalink_sub: string, category_final: string, perma: string, investor_country_code: string, permaround: string, funding_round_type: string, labelacq: string]

In [ ]:
# check for missing values 
dfmaster.toPandas().isnull().sum()

_c0                           0
permalink                     1
name                          1
market                     3967
funding_total_usd             1
status                     1315
country_code               5273
city                       6116
funding_rounds                1
founded_year              10956
quarter_new               10956
age                       10956
permalink_agg                 1
count_investor                1
time_to_first_funding     24731
investor_country_codes    20954
funding_round_types       17110
total_raised_usd          49435
permalink_sub                 1
category_final            45459
perma                     20954
investor_country_code     20954
permaround                17110
funding_round_type        17110
labelacq                      0
dtype: int64

In [ ]:
# drop market columns because of too many level and better breakdown with the category_final column
dfmaster1 = dfmaster.drop("market")

In [ ]:
dfmaster1 = dfmaster1.toPandas()
dfmaster1

,_c0,permalink,name,funding_total_usd,status,country_code,city,funding_rounds,founded_year,quarter_new,...,investor_country_codes,funding_round_types,total_raised_usd,permalink_sub,category_final,perma,investor_country_code,permaround,funding_round_type,labelacq
0,0,/organization/1lay,1Lay,"1,70,000",operating,None,None,1,2013,Q3,...,None,None,None,/organization/1lay,None,None,None,None,None,0
1,1,/organization/24pagebooks,24PageBooks,"50,000",closed,USA,Rochester,1,2010,Q1,...,FRA,seed,None,/organization/24pagebooks,None,/organization/24pagebooks,FRA,/organization/24pagebooks,seed,0
2,2,/organization/5min,5min Media,"1,28,00,000",acquired,USA,New York,3,2007,Q1,...,"USA, USA, USA","venture, venture, venture",None,/organization/5min,None,/organization/5min,USA,/organization/5min,venture,1
3,3,/organization/abpathfinder,ABPathfinder,"9,60,000",operating,USA,Overland Park,3,2010,Q2,...,"USA, USA, USA","venture, venture, venture, venture",None,/organization/abpathfinder,None,/organization/abpathfinder,USA,/organization/abpathfinder,venture,0
4,4,/organization/acid-labs,Acid Labs,-,operating,USA,Santa Monica,1,2003,Q1,...,None,None,None,/organization/acid-labs,None,None,None,None,None,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49440,49440,/organization/yieldmo,Yieldmo,"2,21,00,000",operating,USA,New York,3,2012,Q2,...,"USA, USA, USA, USA, USA, ISR, USA, USA, USA, U...","venture, venture, venture, venture, venture, v...",None,/organization/yieldmo,None,/organization/yieldmo,USA,/organization/yieldmo,venture,0
49441,49441,/organization/youxinpai,Youxinpai,"3,00,00,000",operating,CHN,Beijing,1,2011,Q3,...,"CHN, USA, CHN","venture, venture, venture, venture",None,/organization/youxinpai,None,/organization/youxinpai,CHN,/organization/youxinpai,venture,0
49442,49442,/organization/zeropercent-us,ZeroPercent.us,-,operating,USA,Chicago,1,2011,Q3,...,USA,seed,None,/organization/zeropercent-us,None,/organization/zeropercent-us,USA,/organization/zeropercent-us,seed,0
49443,49443,/organization/zinch,Zinch,"54,86,842",acquired,USA,Orem,3,2007,Q1,...,"USA, USA","venture, venture, angel",None,/organization/zinch,None,/organization/zinch,USA,/organization/zinch,venture,1


In [ ]:
# Replace NaN with mode for categorical variables
dfmaster1['total_raised_usd'] = dfmaster1['total_raised_usd'].fillna(dfmaster1['total_raised_usd'].mode()[0])
dfmaster1['time_to_first_funding'] = dfmaster1['time_to_first_funding'].fillna(dfmaster1['time_to_first_funding'].mode()[0])
dfmaster1['founded_year'] = dfmaster1['founded_year'].fillna(dfmaster1['founded_year'].mode()[0])
dfmaster1['age'] = dfmaster1['age'].fillna(dfmaster1['age'].mode()[0])
dfmaster1['status'] = dfmaster1['status'].fillna(dfmaster1['status'].mode()[0])
dfmaster1['country_code'] = dfmaster1['country_code'].fillna(dfmaster1['country_code'].mode()[0])
dfmaster1['city'] = dfmaster1['city'].fillna(dfmaster1['city'].mode()[0])
dfmaster1['quarter_new'] = dfmaster1['quarter_new'].fillna(dfmaster1['quarter_new'].mode()[0])
dfmaster1['investor_country_codes'] = dfmaster1['investor_country_codes'].fillna(dfmaster1['investor_country_codes'].mode()[0])
dfmaster1['funding_round_types'] = dfmaster1['funding_round_types'].fillna(dfmaster1['funding_round_types'].mode()[0])
dfmaster1['permaround'] = dfmaster1['permaround'].fillna(dfmaster1['permaround'].mode()[0])
dfmaster1['investor_country_code'] = dfmaster1['investor_country_code'].fillna(dfmaster1['investor_country_code'].mode()[0])
dfmaster1['funding_round_type'] = dfmaster1['funding_round_type'].fillna(dfmaster1['funding_round_type'].mode()[0])
dfmaster1['category_final'] = dfmaster1['category_final'].fillna(dfmaster1['category_final'].mode()[0])
dfmaster1['perma'] = dfmaster1['perma'].fillna(dfmaster1['perma'].mode()[0])

In [ ]:
# check for missing values 
dfmaster1.isnull().sum()

_c0                       0
permalink                 1
name                      1
funding_total_usd         1
status                    0
country_code              0
city                      0
funding_rounds            1
founded_year              0
quarter_new               0
age                       0
permalink_agg             1
count_investor            1
time_to_first_funding     0
investor_country_codes    0
funding_round_types       0
total_raised_usd          0
permalink_sub             1
category_final            0
perma                     0
investor_country_code     0
permaround                0
funding_round_type        0
labelacq                  0
dtype: int64

In [ ]:
# drop rows with missing values
dfmaster1drop = dfmaster1.dropna()

In [ ]:
print(dfmaster1drop.count())

_c0                       49444
permalink                 49444
name                      49444
funding_total_usd         49444
status                    49444
country_code              49444
city                      49444
funding_rounds            49444
founded_year              49444
quarter_new               49444
age                       49444
permalink_agg             49444
count_investor            49444
time_to_first_funding     49444
investor_country_codes    49444
funding_round_types       49444
total_raised_usd          49444
permalink_sub             49444
category_final            49444
perma                     49444
investor_country_code     49444
permaround                49444
funding_round_type        49444
labelacq                  49444
dtype: int64


In [ ]:
sql = SQLContext(spark)

/usr/local/lib/python3.8/dist-packages/pyspark/sql/context.py:112: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [ ]:
dfmaster2 = sql.createDataFrame(dfmaster1drop)

In [ ]:
display(dfmaster2)

DataFrame[_c0: string, permalink: string, name: string, funding_total_usd: string, status: string, country_code: string, city: string, funding_rounds: string, founded_year: string, quarter_new: string, age: string, permalink_agg: string, count_investor: string, time_to_first_funding: string, investor_country_codes: string, funding_round_types: string, total_raised_usd: string, permalink_sub: string, category_final: string, perma: string, investor_country_code: string, permaround: string, funding_round_type: string, labelacq: string]

String indexer, one hot encoder and casting to numerics

In [ ]:
# create index for categorical variables
# use pipline to apply indexer
list1 = ["country_code","city","quarter_new","investor_country_code","funding_round_type","category_final"]
indexers = [StringIndexer(inputCol=column, outputCol=column+"_index").fit(dfmaster2) for column in list1]
pipelineindex = Pipeline(stages=indexers).fit(dfmaster2)
dfmasternew = pipelineindex.transform(dfmaster2)

In [ ]:
# convert string to double for numerical variables
dfmasternew = dfmasternew.\
  withColumn("numeric_funding_rounds", dfmasternew["funding_rounds"].cast("int")).\
  withColumn("numeric_age", dfmasternew["age"].cast("int")).\
  withColumn("numeric_count_investor", dfmasternew["count_investor"].cast("int")).\
  withColumn("numeric_time_to_first_funding", dfmasternew["time_to_first_funding"].cast("int")).\
  withColumn("numeric_total_raised_usd", dfmasternew["total_raised_usd"].cast("int")).\
  withColumn("label", dfmasternew["labelacq"].cast("int"))

In [ ]:
dfmasternew = dfmasternew.\
  withColumn("funding_round_type", dfmasternew["funding_round_type"].cast("double")).\
  withColumn("country_code_index", dfmasternew["country_code_index"].cast("double")).\
  withColumn("city_index", dfmasternew["city_index"].cast("double")).\
  withColumn("quarter_new_index", dfmasternew["quarter_new_index"].cast("double")).\
  withColumn("labelacq", dfmasternew["labelacq"].cast("double"))

In [ ]:
# save
dfone = dfmasternew

In [ ]:
display(dfone)

DataFrame[_c0: string, permalink: string, name: string, funding_total_usd: string, status: string, country_code: string, city: string, funding_rounds: string, founded_year: string, quarter_new: string, age: string, permalink_agg: string, count_investor: string, time_to_first_funding: string, investor_country_codes: string, funding_round_types: string, total_raised_usd: string, permalink_sub: string, category_final: string, perma: string, investor_country_code: string, permaround: string, funding_round_type: double, labelacq: double, country_code_index: double, city_index: double, quarter_new_index: double, investor_country_code_index: double, funding_round_type_index: double, category_final_index: double, numeric_funding_rounds: int, numeric_age: int, numeric_count_investor: int, numeric_time_to_first_funding: int, numeric_total_raised_usd: int, label: int]

In [ ]:
print(dfone.count())

49444


In [ ]:
# list of index columns of categorical variables for the onehotencoder
list2 = dfone.columns[24:30]
list2

['country_code_index',
 'city_index',
 'quarter_new_index',
 'investor_country_code_index',
 'funding_round_type_index',
 'category_final_index']

In [ ]:
# create sparse matrix of indexed categorical columns
# use pipline to apply the encoder
onehotencoder_stages = [OneHotEncoder(inputCol=c, outputCol='onehotencoded_' + c) for c in list2]
pipelineonehot = Pipeline(stages=onehotencoder_stages)
pipeline_mode = pipelineonehot.fit(dfone)
df_coded = pipeline_mode.transform(dfone)

In [ ]:
df_coded.show()

+---+--------------------+--------------------+-----------------+---------+------------+-------------+--------------+------------+-----------+----+--------------------+--------------+---------------------+----------------------+--------------------+----------------+--------------------+--------------+--------------------+---------------------+--------------------+------------------+--------+------------------+----------+-----------------+---------------------------+------------------------+--------------------+----------------------+-----------+----------------------+-----------------------------+------------------------+-----+--------------------------------+------------------------+-------------------------------+-----------------------------------------+--------------------------------------+----------------------------------+
|_c0|           permalink|                name|funding_total_usd|   status|country_code|         city|funding_rounds|founded_year|quarter_new| age|       perm


Data split, defining vector assemblers & standard scaler and creating labellist

In [ ]:
# split dataset into training, validaiton and testing dataset
training_df, validation_df, testing_df = df_coded.randomSplit([0.6, 0.3, 0.1])

In [ ]:
training_df.columns[30:35]

['numeric_funding_rounds',
 'numeric_age',
 'numeric_count_investor',
 'numeric_time_to_first_funding',
 'numeric_total_raised_usd']

In [ ]:
training_df.columns[36:42]

['onehotencoded_country_code_index',
 'onehotencoded_city_index',
 'onehotencoded_quarter_new_index',
 'onehotencoded_investor_country_code_index',
 'onehotencoded_funding_round_type_index',
 'onehotencoded_category_final_index']

In [ ]:
# define vector assembler with the features for the modelling
vanum = VectorAssembler(). \
      setInputCols(training_df.columns[30:35]). \
      setOutputCol('features_nonstd')

In [ ]:
# define vector assembler with the features for the modelling
vacate = VectorAssembler(). \
      setInputCols(training_df.columns[36:42]). \
      setOutputCol('featurescate')

In [ ]:
va = VectorAssembler(). \
      setInputCols(['featuresnum','featurescate']). \
      setOutputCol('features')

In [ ]:
std = feature.StandardScaler(withMean=True, withStd=True).setInputCol('features_nonstd').setOutputCol('featuresnum')

In [ ]:
# suffix for investor country code because intersection with county_code of  the companies
invcc = ['{}_{}'.format(a, "investor") for a in indexers[3].labels]

In [ ]:
# define labellist by using the indexer stages for displaying the weights & loadings
labellist = training_df.columns[30:35] + indexers[0].labels + indexers[1].labels + indexers[2].labels + invcc + indexers[4].labels + indexers[5].labels

In [ ]:
# null dummy for onehotencoded_country_code_index
print("null dummy for onehotencoded_country_code_index")
print(len(indexers[0].labels))
print(indexers[0].labels)
# null dummy for onehotencoded_city_index
print("null dummy for onehotencoded_city_index")
print(indexers[1].labels)
print(len(indexers[1].labels))
# null dummy for onehotencoded_quarter_new_index
print("null dummy for onehotencoded_quarter_new_index")
print(len(indexers[2].labels))
print(indexers[2].labels)
# null dummy for onehotencoded_investor_country_code_index
print("null dummy for onehotencoded_investor_country_code_index")
print(len(invcc))
print(invcc)
# null dummy for onehotencoded_funding_round_type_index
print("null dummy for onehotencoded_funding_round_type_index")
print(len(indexers[4].labels))
print(indexers[4].labels)
# null dummy for onehotencoded_category_final_index
print("null dummy for onehotencoded_category_final_index")
print(len(indexers[5].labels))
print(indexers[5].labels)

null dummy for onehotencoded_country_code_index
115
['USA', 'GBR', 'CAN', 'CHN', 'DEU', 'FRA', 'IND', 'ISR', 'ESP', 'RUS', 'SWE', 'AUS', 'ITA', 'NLD', 'IRL', 'SGP', 'BRA', 'CHL', 'JPN', 'KOR', 'CHE', 'DNK', 'FIN', 'ARG', 'BEL', 'HKG', 'TUR', 'AUT', 'NOR', 'POL', 'MEX', 'PRT', 'BGR', 'ARE', 'NZL', 'IDN', 'ZAF', 'CZE', 'MYS', 'UKR', 'EST', 'HUN', 'TWN', 'THA', 'COL', 'PHL', 'GRC', 'LTU', 'PER', 'NGA', 'KEN', 'EGY', 'LUX', 'ROM', 'VNM', 'DZA', 'JOR', 'PAK', 'ISL', 'SVK', 'LBN', 'CYP', 'LVA', 'URY', 'CYM', 'GHA', 'SVN', 'KHM', 'SRB', 'UGA', 'HRV', 'BGD', 'SAU', 'TZA', 'CRI', 'PAN', 'BMU', 'BWA', 'GTM', 'AZE', 'BHR', 'BLR', 'DOM', 'MAR', 'MLT', 'SLV', 'TUN', 'ARM', 'BHS', 'CMR', 'ECU', 'GIB', 'KWT', 'LAO', 'MDA', 'MKD', 'MMR', 'NIC', 'NPL', 'ALB', 'BRN', 'CIV', 'JAM', 'JEY', 'LIE', 'MAF', 'MCO', 'MOZ', 'MUS', 'OMN', 'SOM', 'SYC', 'TTO', 'UZB', 'ZWE']
null dummy for onehotencoded_city_index
['San Francisco', 'New York', 'London', 'Palo Alto', 'Austin', 'Cambridge', 'Seattle', 'Chicago', 'Los

# Modeling

## DECISION TREE

In [ ]:
# define multiclass classification evaluator
mce = MulticlassClassificationEvaluator()

In [ ]:
dt = DecisionTreeClassifier(labelCol="label", featuresCol="features")

In [ ]:
dt_pipeline = Pipeline(stages=[vanum, std, vacate, va, dt]).fit(training_df)

In [ ]:
dfdt = dt_pipeline.transform(validation_df)

In [ ]:
# print the areas under the curve for the different random forest pipelines
print("Decision Tree: AUC = {}".format(mce.evaluate(dfdt)))

Decision Tree: AUC = 0.8889313652753997


In [ ]:
# print the accuracies for the different random forest pipelines
print(dfdt.select(fn.expr('float(label = prediction)').alias('correct')).select(fn.avg('correct').alias("Accuracy for Decision Tree")).show())

+--------------------------+
|Accuracy for Decision Tree|
+--------------------------+
|        0.9247161919936268|
+--------------------------+

None


## RANDOM FOREST

In [ ]:
# define binary classification evaluator
bce = BinaryClassificationEvaluator()

In [ ]:
# define default, 15 trees and 25 trees random forest classifier
rf = RandomForestClassifier(maxBins=10000, featuresCol='features', labelCol='label')
rf15 = RandomForestClassifier(numTrees=15, maxBins=10000, featuresCol='features', labelCol='label')
rf25 = RandomForestClassifier(numTrees=25, maxBins=10000, featuresCol='features', labelCol='label')

In [ ]:
# define and fit pipelines with vector assembler and random forest classifier 
rf_pipeline = Pipeline(stages=[vanum, std, vacate, va, rf]).fit(training_df)
rf_pipeline_15 = Pipeline(stages=[vanum, std, vacate, va, rf15]).fit(training_df)
rf_pipeline_25 = Pipeline(stages=[vanum, std, vacate, va, rf25]).fit(training_df)

In [ ]:
dfrf = rf_pipeline.transform(validation_df)
dfrf_15 = rf_pipeline_15.transform(validation_df)
dfrf_25 = rf_pipeline_25.transform(validation_df)

In [ ]:
dfrf.show()

+-----+--------------------+--------------------+-----------------+---------+------------+-------------+--------------+------------+-----------+----+--------------------+--------------+---------------------+----------------------+--------------------+----------------+--------------------+--------------+--------------------+---------------------+--------------------+------------------+--------+------------------+----------+-----------------+---------------------------+------------------------+--------------------+----------------------+-----------+----------------------+-----------------------------+------------------------+-----+--------------------------------+------------------------+-------------------------------+-----------------------------------------+--------------------------------------+----------------------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|  _c0|           p

## Performance

In [ ]:
# print the areas under the curve for the different random forest pipelines
print("Random Forest with 20 trees: AUC = {}".format(bce.evaluate(dfrf)))
print("Random Forest 15 trees: AUC = {}".format(bce.evaluate(dfrf_15)))
print("Random Forest 25 trees: AUC = {}".format(bce.evaluate(dfrf_25)))

Random Forest with 20 trees: AUC = 0.6704048578881618
Random Forest 15 trees: AUC = 0.7081371405125284
Random Forest 25 trees: AUC = 0.6837994917522479


In [ ]:
# print the accuracies for the different random forest pipelines
print(dfrf.select(fn.expr('float(label = prediction)').alias('correct')).select(fn.avg('correct').alias("Accuracy for Random Forest with 20 trees")).show())
print(dfrf_15.select(fn.expr('float(label = prediction)').alias('correct')).select(fn.avg('correct').alias("Accuracy for Random Forest with 15 trees")).show())
print(dfrf_25.select(fn.expr('float(label = prediction)').alias('correct')).select(fn.avg('correct').alias("Accuracy for Random Forest with 25 trees")).show())

+----------------------------------------+
|Accuracy for Random Forest with 20 trees|
+----------------------------------------+
|                      0.9249817433446192|
+----------------------------------------+

None
+----------------------------------------+
|Accuracy for Random Forest with 15 trees|
+----------------------------------------+
|                      0.9249817433446192|
+----------------------------------------+

None
+----------------------------------------+
|Accuracy for Random Forest with 25 trees|
+----------------------------------------+
|                      0.9249817433446192|
+----------------------------------------+

None


## Importancies and Weights

In [ ]:
# create spark df with the 20 highest labels and the corresponding importancies + sorting by importancy
rfw = spark.createDataFrame(pd.DataFrame(list(zip(labellist, rf_pipeline.stages[4].featureImportances.toArray())),
            columns = ['column', 'importancy']).sort_values('importancy').tail(20))
display(rfw)
rfw.show()

DataFrame[column: string, importancy: double]

+--------------------+--------------------+
|              column|          importancy|
+--------------------+--------------------+
|           Blue Bell|0.015905884160977338|
|            Adelaide|0.016227370831204124|
| Newcastle Upon Tyne|0.017841583609301697|
|              London|0.020912708374023172|
|            Limerick|0.022439994687994978|
|        MEX_investor|0.022446794445075937|
|            Prospect| 0.02343734706764813|
|                 CHN|0.024097968920304323|
|          Wollongong| 0.02490325369848877|
|     West Palm Beach|0.026049973350257414|
|        KWT_investor| 0.02862139175931437|
|          Des Moines| 0.03028230975604472|
|numeric_funding_r...|0.032848588310300963|
|    Bloomfield Hills| 0.03391971890954374|
|                 USA| 0.03669846419205969|
|        MAR_investor| 0.04011345835101589|
|          Southfield| 0.04777590052850689|
|        Natchitoches|0.055555555555555566|
|         Los Angeles| 0.07275190515374169|
|numeric_time_to_f...|  0.072980

In [ ]:
# create spark df with the labels and the corresponding importancies + sorting by importancy
rf15w = spark.createDataFrame(pd.DataFrame(list(zip(labellist, rf_pipeline_15.stages[4].featureImportances.toArray())),
            columns = ['column', 'importancy']).sort_values('importancy').tail(20))
display(rf15w)
rf15w.show()

DataFrame[column: string, importancy: double]

+--------------------+--------------------+
|              column|          importancy|
+--------------------+--------------------+
|          Bedminster|  0.0155651704194964|
|                Jena|0.016193225801187282|
|        USA_investor| 0.01755285687358526|
|          Carrollton| 0.01941443016032135|
|     Fort Washington|0.023282041475063565|
|              Yangon| 0.02333638441691831|
|             Sanford|  0.0257912812607462|
|            San Jose|0.029381420565460194|
|              Nagpur|0.031026842819954267|
|                 CHN| 0.03138271628714425|
|                  Q1|0.031953304152320575|
|        Natchitoches| 0.03334327291390294|
|              Rennes|0.033755364243386564|
|         Los Angeles|0.035740350838769336|
|numeric_time_to_f...|0.054176021048508695|
|numeric_count_inv...|0.055457496924461426|
|              London| 0.05882655811974447|
|           Weybridge| 0.06305259075919441|
|          Des Moines| 0.07142857142857142|
|        MAR_investor| 0.0833057

In [ ]:
# create spark df with the labels and the corresponding importancies + sorting by importancy
rf25w = spark.createDataFrame(pd.DataFrame(list(zip(labellist, rf_pipeline_25.stages[4].featureImportances.toArray())),
            columns = ['column', 'weight']).sort_values('weight').tail(20))
display(rf25w)
rf25w.show()

DataFrame[column: string, weight: double]

+--------------------+--------------------+
|              column|              weight|
+--------------------+--------------------+
|    Westlake Village|0.014402208566303674|
|           Guangdong|0.014493730322552066|
|          Bedminster|0.015308414120837164|
| Newcastle Upon Tyne| 0.01570869938699635|
|    Bloomfield Hills|0.016652467220931107|
|           Weybridge|0.016840288781024945|
|               Yavne|0.017263562868494498|
|           Fullerton|0.017373442321439888|
|numeric_time_to_f...| 0.01788824775559418|
|       North Andover| 0.01807185084664363|
|          Southfield|0.019693354024959094|
|            San Jose| 0.02006838440705003|
|         Glastonbury|0.025406410022921596|
|          Carrollton|0.026825793648634126|
|               Tampa|0.027795708777878132|
|              London|0.040805882660936994|
|          Ridgefield| 0.04129539779563641|
|numeric_count_inv...| 0.04248324763240757|
|               Alamo|0.057321286185158866|
|         Los Angeles| 0.0711345

# TESTING PERFORMANCE

We tested with the best performing (AUC and Accuracy) Decision Tree and Random Forest model.

In [ ]:
# Decision Tree
dfdt_test = dt_pipeline.transform(testing_df)
print("Decision Tree: AUC = {}".format(mce.evaluate(dfdt_test)))
print(dfdt_test.select(fn.expr('float(label = prediction)').alias('correct')).select(fn.avg('correct').alias("Accuracy for Decision Tree")).show())
dt_true = dfdt_test.select(['label']).collect()
dt_pred = dfdt_test.select(['prediction']).collect()
print(classification_report(dt_true, dt_pred))

Decision Tree: AUC = 0.8842540957046967
+--------------------------+
|Accuracy for Decision Tree|
+--------------------------+
|        0.9211061407076048|
+--------------------------+

None
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      4535
           1       0.00      0.00      0.00       383

    accuracy                           0.92      4918
   macro avg       0.46      0.50      0.48      4918
weighted avg       0.85      0.92      0.88      4918



In [ ]:
# Best performing random forest model
dfrf_25_test = rf_pipeline_25.transform(testing_df)
print("Random Forest 25 trees: AUC = {}".format(bce.evaluate(dfrf_25_test)))
print(dfrf_25_test.select(fn.expr('float(label = prediction)').alias('correct')).select(fn.avg('correct').alias("Accuracy for Random Forest with 25 trees")).show())
rf_true = dfrf_25_test.select(['label']).collect()
rf_pred = dfrf_25_test.select(['prediction']).collect()
print(classification_report(rf_true, rf_pred))

Random Forest 25 trees: AUC = 0.6806307771582212
+----------------------------------------+
|Accuracy for Random Forest with 25 trees|
+----------------------------------------+
|                      0.9221228141520943|
+----------------------------------------+

None
              precision    recall  f1-score   support

           0       0.92      1.00      0.96      4535
           1       0.00      0.00      0.00       383

    accuracy                           0.92      4918
   macro avg       0.46      0.50      0.48      4918
weighted avg       0.85      0.92      0.88      4918



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


#Logistic Regression

In [ ]:
# define logistic regression parameters and pipeline stages 
lambda_par1 = 0.01
alpha_par1 = 0.05
en_lr1 = LogisticRegression().\
        setLabelCol('label').\
        setFeaturesCol('features').\
        setRegParam(lambda_par1).\
        setElasticNetParam(alpha_par1)

# change the parameters of the second classifier below
lambda_par2 = 0.05
alpha_par2 = 0.05
en_lr2 = LogisticRegression().\
        setLabelCol('label').\
        setFeaturesCol('features').\
        setRegParam(lambda_par2).\
        setElasticNetParam(alpha_par2)

# change the parameters of the thrid classifier below
lambda_par3 = 0.1
alpha_par3 = 0.05
en_lr3 = LogisticRegression().\
        setLabelCol('label').\
        setFeaturesCol('features').\
        setRegParam(lambda_par3).\
        setElasticNetParam(alpha_par3)
        
en_lr_estimator1 = Pipeline(
    stages=[vanum, std, vacate, va, en_lr1])
en_lr_estimator2 = Pipeline(
    stages=[vanum, std, vacate, va, en_lr2])
en_lr_estimator3 = Pipeline(
    stages=[vanum, std, vacate, va, en_lr3])

In [ ]:
# fit logistic regression pipelines
en_lr_model1 = en_lr_estimator1.fit(training_df)
en_lr_model2 = en_lr_estimator2.fit(training_df)
en_lr_model3 = en_lr_estimator3.fit(training_df)

In [ ]:
# apply pipeline to validation dataset
dfmodel1 = en_lr_model1.transform(validation_df)
dfmodel2 = en_lr_model2.transform(validation_df)
dfmodel3 = en_lr_model3.transform(validation_df)

In [ ]:
# define binary classification evaluator
bce = BinaryClassificationEvaluator()

In [ ]:
# print areas under the curve of the different logistic regressions
print("Logistic Regression Model 1: AUC = {}".format(bce.evaluate(dfmodel1)))
print("Logistic Regression Model 2: AUC = {}".format(bce.evaluate(dfmodel2)))
print("Logistic Regression Model 3: AUC = {}".format(bce.evaluate(dfmodel3)))

Logistic Regression Model 1: AUC = 0.7109942398164669
Logistic Regression Model 2: AUC = 0.7283555180957673
Logistic Regression Model 3: AUC = 0.7323500456355924


In [ ]:
# print accuracies of the different logistic regressions
print(dfmodel1.select(fn.expr('float(label = prediction)').alias('correct')).select(fn.avg('correct').alias("Accuracy for Model 1")).show())
print(dfmodel2.select(fn.expr('float(label = prediction)').alias('correct')).select(fn.avg('correct').alias("Accuracy for Model 2")).show())
print(dfmodel3.select(fn.expr('float(label = prediction)').alias('correct')).select(fn.avg('correct').alias("Accuracy for Model 3")).show())

+--------------------+
|Accuracy for Model 1|
+--------------------+
|  0.9197371041625174|
+--------------------+

None
+--------------------+
|Accuracy for Model 2|
+--------------------+
|  0.9241187014538936|
+--------------------+

None
+--------------------+
|Accuracy for Model 3|
+--------------------+
|  0.9249817433446192|
+--------------------+

None


In [ ]:
dfdislr = spark.createDataFrame(pd.DataFrame(list(zip(labellist, en_lr_model1.stages[4].coefficients.toArray())),
            columns = ['label', 'weight']).sort_values('weight')).agg((fn.count(fn.when(col("weight") == 0, True)) / fn.count(col("label"))).alias("Ratio"))
dfdislr.show()

+------------------+
|             Ratio|
+------------------+
|0.7610820641054785|
+------------------+



#Weights

In [ ]:
# create spark df with the first 10 lowest labels and corresponding weights
pd.DataFrame(list(zip(labellist, en_lr_model1.stages[4].coefficients.toArray())),
            columns = ['label', 'weight']).sort_values('weight').head(10)

,label,weight
3485,Nordborg,-3.269576
2242,Anth�e,-3.040357
3325,Matthews,-3.010850
1882,Logro�o,-2.855047
840,Southlake,-2.832794
3863,Schagen,-2.719455
1885,Loughton,-2.659051
3941,Sopot,-2.408851
2536,Cheraw,-2.354156
613,Lisle,-2.268032


In [ ]:
# create spark df with the first 10 highest labels and corresponding weights
pd.DataFrame(list(zip(labellist, en_lr_model1.stages[4].coefficients.toArray())),
            columns = ['label', 'weight']).sort_values('weight').tail(10)

,label,weight
4050,Temple Hills,5.463621
3278,Mafra,5.465486
3753,Roland,5.468296
1433,Nagpur,5.471445
2025,Ridgefield,5.592470
2179,Wollongong,5.693179
2748,Essex Junction,5.759650
633,Bloomfield Hills,5.842307
3962,Spicewood,5.966595
2610,Cottage Grove,6.529684


In [ ]:
# create spark df with the first 10 lowest labels and corresponding weights
pd.DataFrame(list(zip(labellist, en_lr_model2.stages[4].coefficients.toArray())),
            columns = ['label', 'weight']).sort_values('weight').head(10)

,label,weight
4312,USA_investor,-0.265318
4392,undisclosed,-0.265008
8,CHN,-0.222427
4319,CHL_investor,-0.179162
3325,Matthews,-0.177998
4387,MAR_investor,-0.170020
4316,DEU_investor,-0.160062
11,IND,-0.158828
4321,JPN_investor,-0.138544
3485,Nordborg,-0.130563


In [ ]:
# create spark df with the first 10 highest labels and corresponding weights
pd.DataFrame(list(zip(labellist, en_lr_model2.stages[4].coefficients.toArray())),
            columns = ['label', 'weight']).sort_values('weight').tail(10)

,label,weight
1433,Nagpur,2.757488
2168,Weybridge,2.763019
4020,Sylvania,2.771838
4050,Temple Hills,2.772583
3278,Mafra,2.791841
2179,Wollongong,2.814908
1626,Brecksville,2.830611
2748,Essex Junction,2.916315
2610,Cottage Grove,2.969507
1262,Aptos,3.032207


In [ ]:
# create spark df with the first lowest labels and corresponding weights
pd.DataFrame(list(zip(labellist, en_lr_model3.stages[4].coefficients.toArray())),
            columns = ['label', 'weight']).sort_values('weight').head(10)

,label,weight
4387,MAR_investor,-0.112941
8,CHN,-0.103347
4312,USA_investor,-0.101313
11,IND,-0.030580
4392,undisclosed,-0.009593
4316,DEU_investor,-0.007125
2916,Hailsham,0.000000
2915,Haidershofen,0.000000
2914,Hagerman,0.000000
2913,Hafnarfj�r�ur,0.000000


In [ ]:
# create spark df with the first 10 highest labels and corresponding weights
pd.DataFrame(list(zip(labellist, en_lr_model3.stages[4].coefficients.toArray())),
            columns = ['label', 'weight']).sort_values('weight').tail(10)

,label,weight
3278,Mafra,0.530684
2179,Wollongong,0.539024
2748,Essex Junction,0.578040
1626,Brecksville,0.590771
714,Utrecht,0.606331
517,Carrollton,0.656092
543,Des Moines,0.700233
963,Darmstadt,1.161160
2168,Weybridge,1.397489
1262,Aptos,1.549943


Weights of the important features of the Random Forest model with 15 trees as reference

In [ ]:
# create spark df with all coefficients of LR model 2 (best performance in terms of AUC and accuracy combined)
dflrcoeff = spark.createDataFrame(pd.DataFrame(list(zip(labellist, en_lr_model2.stages[4].coefficients.toArray())),
            columns = ['label', 'weight']).sort_values('weight'))

In [ ]:
# list with the highest 8 importancies based on RF model with 15 trees (best performance in terms of AUC and accuracy combined)
rf15imp = ["Storage","Messaging","numeric_count_investor","USA","venture","USA_investor","numeric_age","numeric_total_raised_usd"]

In [ ]:
# display weights for the important factors from the RF model with 15 trees in order to see if its negative or positive
dfdisrf = dflrcoeff.where(col("label").isin(rf15imp)).orderBy("weight")
dfdisrf.show()

+--------------------+--------------------+
|               label|              weight|
+--------------------+--------------------+
|        USA_investor|-0.26531770618420764|
|             venture|                 0.0|
|numeric_total_rai...|                 0.0|
|         numeric_age| 0.17796237042371646|
|numeric_count_inv...| 0.22518685761219065|
|                 USA|  0.2455042371907676|
+--------------------+--------------------+



#Principal Component Analysis

In [ ]:
# define and fit a PCA model wit k = 200 as reference

pcavar = feature.PCA(k=200, inputCol='features', outputCol='pca_feat')
pca_var = Pipeline(stages=[vanum, std, vacate, va, pcavar])
pca_var_model = pca_var.fit(df_coded)
varlist = pca_var_model.stages[4].explainedVariance
npvar = np.cumsum(varlist)
pci = [pci for pci in range(1,201,1)]
dfvar = spark.createDataFrame(pd.DataFrame({"Number of PCs": pci, "Cumulative Variance Explained": npvar}))
dfvar.show()

+-------------+-----------------------------+
|Number of PCs|Cumulative Variance Explained|
+-------------+-----------------------------+
|            1|          0.22833416514833127|
|            2|           0.4245649216190356|
|            3|           0.5542144510065699|
|            4|           0.6195314423589732|
|            5|           0.6656582494370347|
|            6|            0.703573108899761|
|            7|           0.7381980723011159|
|            8|           0.7690787356392836|
|            9|           0.7875119527317764|
|           10|           0.7993796580257001|
|           11|           0.8106491495894524|
|           12|           0.8203493854932905|
|           13|           0.8291191210214209|
|           14|           0.8345718391025445|
|           15|           0.8392957595652807|
|           16|           0.8436070497972814|
|           17|           0.8477596808439594|
|           18|           0.8516652239101569|
|           19|           0.854982

In [ ]:
dfvar.show(dfvar.count(), False)

+-------------+-----------------------------+
|Number of PCs|Cumulative Variance Explained|
+-------------+-----------------------------+
|1            |0.22833416514833127          |
|2            |0.4245649216190356           |
|3            |0.5542144510065699           |
|4            |0.6195314423589732           |
|5            |0.6656582494370347           |
|6            |0.703573108899761            |
|7            |0.7381980723011159           |
|8            |0.7690787356392836           |
|9            |0.7875119527317764           |
|10           |0.7993796580257001           |
|11           |0.8106491495894524           |
|12           |0.8203493854932905           |
|13           |0.8291191210214209           |
|14           |0.8345718391025445           |
|15           |0.8392957595652807           |
|16           |0.8436070497972814           |
|17           |0.8477596808439594           |
|18           |0.8516652239101569           |
|19           |0.8549825054333559 

**So we reach a explained variance of 90% with roughly 85 Principal Components**

In [ ]:
# define pca feature function with k = 85
pca = feature.PCA(k=85, inputCol='features', outputCol='pca_feat')

In [ ]:
# define the parameters and pipelines for the logistic regressions with PCA features
lambda_par1 = 0.01
alpha_par1 = 0.05
en_lr1_pca = LogisticRegression().\
        setLabelCol('label').\
        setFeaturesCol('pca_feat').\
        setRegParam(lambda_par1).\
        setElasticNetParam(alpha_par1)

# change the parameters of the second classifier below
lambda_par2 = 0.05
alpha_par2 = 0.05
en_lr2_pca = LogisticRegression().\
        setLabelCol('label').\
        setFeaturesCol('pca_feat').\
        setRegParam(lambda_par2).\
        setElasticNetParam(alpha_par2)

# change the parameters of the thrid classifier below
lambda_par3 = 0.1
alpha_par3 = 0.05
en_lr3_pca = LogisticRegression().\
        setLabelCol('label').\
        setFeaturesCol('pca_feat').\
        setRegParam(lambda_par3).\
        setElasticNetParam(alpha_par3)
        
en_lr_estimator1_pca = Pipeline(
    stages=[vanum, std, vacate, va, pca, en_lr1_pca])
en_lr_estimator2_pca = Pipeline(
    stages=[vanum, std, vacate, va, pca, en_lr2_pca])
en_lr_estimator3_pca = Pipeline(
    stages=[vanum, std, vacate, va, pca, en_lr3_pca])

In [ ]:
# fit the PCA logistic regression pipelines
en_lr_model1_pca = en_lr_estimator1_pca.fit(training_df)
en_lr_model2_pca = en_lr_estimator2_pca.fit(training_df)
en_lr_model3_pca = en_lr_estimator3_pca.fit(training_df)

In [ ]:
# apply the pipelines to the validation dataset
dfmodel1_pca = en_lr_model1_pca.transform(validation_df)
dfmodel2_pca = en_lr_model2_pca.transform(validation_df)
dfmodel3_pca = en_lr_model3_pca.transform(validation_df)

In [ ]:
# print the areas under the curve and accuracies of the different PCA logistic regression models
print("PC Logistic Regression Model 1: AUC = {}".format(bce.evaluate(dfmodel1_pca)))
print("PC Logistic Regression Model 2: AUC = {}".format(bce.evaluate(dfmodel2_pca)))
print("PC Logistic Regression Model 3: AUC = {}".format(bce.evaluate(dfmodel3_pca)))
#Accuracy
print(dfmodel1_pca.select(fn.expr('float(label = prediction)').alias('correct')).select(fn.avg('correct').alias("Accuracy for Model 1")).show())
print(dfmodel2_pca.select(fn.expr('float(label = prediction)').alias('correct')).select(fn.avg('correct').alias("Accuracy for Model 2")).show())
print(dfmodel3_pca.select(fn.expr('float(label = prediction)').alias('correct')).select(fn.avg('correct').alias("Accuracy for Model 3")).show())

PC Logistic Regression Model 1: AUC = 0.7250762022295071
PC Logistic Regression Model 2: AUC = 0.7352234048026305
PC Logistic Regression Model 3: AUC = 0.7376471088883656
+--------------------+
|Accuracy for Model 1|
+--------------------+
|  0.9234548230764124|
+--------------------+

None
+--------------------+
|Accuracy for Model 2|
+--------------------+
|  0.9244506406426343|
+--------------------+

None
+--------------------+
|Accuracy for Model 3|
+--------------------+
|  0.9249817433446192|
+--------------------+

None


Exploring Loadings

In [ ]:
# assign loadings of the first 3 PCs 
pc1 = en_lr_model2_pca.stages[4].pc.toArray()[:, 0].tolist()
pc2 = en_lr_model2_pca.stages[4].pc.toArray()[:, 1].tolist()
pc3 = en_lr_model2_pca.stages[4].pc.toArray()[:, 2].tolist()

In [ ]:
# create pandas df with the loadings/PCs and the corresponding labels
pc_loadings = pd.DataFrame([labellist, pc1, pc2, pc3]).T.rename(columns={0: 'label', 
                                                                          1: 'load_pc1',
                                                                          2: 'load_pc2',
                                                                          3: 'load_pc3',})

In [ ]:
# create spark df with the 5 highest and lowest loadings of the first PC
load1 = sql.createDataFrame(pd.concat((pc_loadings.sort_values('load_pc1').head(), 
           pc_loadings.sort_values('load_pc1').tail())))
display(load1)
load1.show()

DataFrame[label: string, load_pc1: double, load_pc2: double, load_pc3: double]

+--------------------+--------------------+--------------------+--------------------+
|               label|            load_pc1|            load_pc2|            load_pc3|
+--------------------+--------------------+--------------------+--------------------+
|         numeric_age| -0.6036457658329708|  -0.363973967219413|-0.00410482576020...|
|numeric_time_to_f...| -0.5859227874086506| -0.3854471979163721|-0.00605110074920...|
|numeric_funding_r...| -0.3833252853181104|  0.5940044857574776|0.005551606078036...|
|numeric_count_inv...| -0.3691400520240835|  0.6006682593786625|0.019919209280200287|
|        �sterby Havn|-0.05005800871880814|-0.03978762053326017|-0.00366214207883...|
|       post_ipo_debt|                null|                null|                null|
|product_crowdfunding|                null|                null|                null|
|    secondary_market|                null|                null|                null|
|                None|                null|           

Here we can see the top and bottom 5 features based on the loadings that describe the first PC.

In [ ]:
# create spark df with the 5 highest and lowest loadings of the second PC
load2 = spark.createDataFrame(pd.concat((pc_loadings.sort_values('load_pc2').head(), 
           pc_loadings.sort_values('load_pc2').tail())))
display(load2)
load2.show()

DataFrame[label: string, load_pc1: double, load_pc2: double, load_pc3: double]

+--------------------+--------------------+--------------------+--------------------+
|               label|            load_pc1|            load_pc2|            load_pc3|
+--------------------+--------------------+--------------------+--------------------+
|numeric_time_to_f...| -0.5859227874086506| -0.3854471979163721|-0.00605110074920...|
|         numeric_age| -0.6036457658329708|  -0.363973967219413|-0.00410482576020...|
|        �sterby Havn|-0.05005800871880814|-0.03978762053326017|-0.00366214207883...|
|        TTO_investor|-0.00773227749263...|-0.02026095114825806|-6.63623369231980...|
|             venture|-0.01131975506444...|-0.00785273104516...|-1.69653022860796...|
|       post_ipo_debt|                null|                null|                null|
|product_crowdfunding|                null|                null|                null|
|    secondary_market|                null|                null|                null|
|                None|                null|           

Here we can see the top and bottom 5 features based on the loadings that describe the second PC.:

In [ ]:
# create spark df with the 5 highest and lowest loadings of the third PC
load3 = spark.createDataFrame(pd.concat((pc_loadings.sort_values('load_pc3').head(), 
           pc_loadings.sort_values('load_pc3').tail())))
display(load3)
load3.show()

DataFrame[label: string, load_pc1: double, load_pc2: double, load_pc3: double]

+--------------------+--------------------+--------------------+--------------------+
|               label|            load_pc1|            load_pc2|            load_pc3|
+--------------------+--------------------+--------------------+--------------------+
|numeric_total_rai...|-0.00403050828910...|0.019498915601626966| -0.9996038865720893|
|                 GBR|8.148557208618151E-4|-0.00468550763395...|-0.00736443893001...|
|                  Q3|-0.00454103489453...|-0.00506841095817...|-0.00624778861036...|
|numeric_time_to_f...| -0.5859227874086506| -0.3854471979163721|-0.00605110074920...|
|        MAR_investor|0.055245705060509194|0.028755161060297782|-0.00523800173251...|
|       post_ipo_debt|                null|                null|                null|
|product_crowdfunding|                null|                null|                null|
|    secondary_market|                null|                null|                null|
|                None|                null|           

Here we can see the top and bottom 5 features based on the loadings that describe the third PC.

#Testing Performance

In [ ]:
# Best performing logistic regression model with PCA
dfmodel2_pca_test = en_lr_model2_pca.transform(testing_df)
print("PC Logistic Regression Model 2: AUC = {}".format(bce.evaluate(dfmodel2_pca_test)))
print(dfmodel2_pca_test.select(fn.expr('float(label = prediction)').alias('correct')).select(fn.avg('correct').alias("Accuracy for PC-LR Model 2")).show())

PC Logistic Regression Model 2: AUC = 0.7547027615212122
+--------------------------+
|Accuracy for PC-LR Model 2|
+--------------------------+
|        0.9221228141520943|
+--------------------------+

None


In [ ]:
lr_true = dfmodel2_pca_test.select(['label']).collect()
lr_pred = dfmodel2_pca_test.select(['prediction']).collect()
from sklearn.metrics import classification_report, confusion_matrix
print(classification_report(lr_true, lr_pred))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96      4535
           1       0.50      0.01      0.01       383

    accuracy                           0.92      4918
   macro avg       0.71      0.50      0.48      4918
weighted avg       0.89      0.92      0.89      4918



#Neural Networks

###Standard Model

In [ ]:
# define neural networks (MultilayerPerceptron) classifier
mlp = classification.MultilayerPerceptronClassifier(seed=0).\
    setStepSize(0.2).\
    setMaxIter(200).\
    setLayers([4399, 2]).\
    setFeaturesCol('features')

In [ ]:
# define and fit neural network pipeline 
mlp_simple_model = Pipeline(stages=[vanum, std, vacate, va, mlp]).fit(training_df)

In [ ]:
# define evaluators for accuracy and area under the curve
evaluator = MulticlassClassificationEvaluator().setMetricName("accuracy")
evaluatorauc = MulticlassClassificationEvaluator()

In [ ]:
# apply pipeline to validation dataset
dfnn = mlp_simple_model.transform(validation_df)

In [ ]:
# print accuracy and area under the curve for NN
print(evaluator.evaluate(dfnn))
print(evaluatorauc.evaluate(dfnn))

0.9170152028148443
0.8863205396481113


###Model with Hidden Layers

In [ ]:
# define neural networks (MultilayerPerceptron) classifier with hidden layers
mlp2 = classification.MultilayerPerceptronClassifier(seed=0).\
    setStepSize(0.2).\
    setMaxIter(200).\
    setFeaturesCol('features').\
    setLayers([4399,30,30, 2])
mlp3 = classification.MultilayerPerceptronClassifier(seed=0).\
    setStepSize(0.2).\
    setMaxIter(200).\
    setFeaturesCol('features').\
    setLayers([4399,10,10, 2])
mlp4 = classification.MultilayerPerceptronClassifier(seed=0).\
    setStepSize(0.2).\
    setMaxIter(200).\
    setFeaturesCol('features').\
    setLayers([4399,20,20, 2])
mlp5 = classification.MultilayerPerceptronClassifier(seed=0).\
    setStepSize(0.2).\
    setMaxIter(200).\
    setFeaturesCol('features').\
    setLayers([4399,30, 2])
mlp6 = classification.MultilayerPerceptronClassifier(seed=0).\
    setStepSize(0.2).\
    setMaxIter(200).\
    setFeaturesCol('features').\
    setLayers([4399,30,30,30, 2])

In [ ]:
# define and fit pipeline
mlp2_model = Pipeline(stages=[vanum, std, vacate, va, mlp2]).fit(training_df)
mlp3_model = Pipeline(stages=[vanum, std, vacate, va, mlp3]).fit(training_df)
mlp4_model = Pipeline(stages=[vanum, std, vacate, va, mlp4]).fit(training_df)
mlp5_model = Pipeline(stages=[vanum, std, vacate, va, mlp5]).fit(training_df)
mlp6_model = Pipeline(stages=[vanum, std, vacate, va, mlp6]).fit(training_df)

In [ ]:
# apply and fit model to validation dataset
dfnn2 = mlp2_model.transform(validation_df)
dfnn3 = mlp3_model.transform(validation_df)
dfnn4 = mlp4_model.transform(validation_df)
dfnn5 = mlp5_model.transform(validation_df)
dfnn6 = mlp6_model.transform(validation_df)

###Performance

In [ ]:
# print accuracy and area under the curve
print("NN Model with 2 hidden layers and 30 neurons each: Accuracy = {}".format(evaluator.evaluate(dfnn2)))
print("NN Model with 2 hidden layers and 30 neurons each: AUC = {}".format(evaluatorauc.evaluate(dfnn2)))
print("____________________________________________________________________________")
print("NN Model with 2 hidden layers and 10 neurons each: Accuracy = {}".format(evaluator.evaluate(dfnn3)))
print("NN Model with 2 hidden layers and 10 neurons each: AUC = {}".format(evaluatorauc.evaluate(dfnn3)))
print("____________________________________________________________________________")
print("NN Model with 2 hidden layers and 20 neurons each: Accuracy = {}".format(evaluator.evaluate(dfnn4)))
print("NN Model with 2 hidden layers and 20 neurons each: AUC = {}".format(evaluatorauc.evaluate(dfnn4)))
print("____________________________________________________________________________")
print("NN Model with 1 hidden layer and 30 neurons: Accuracy = {}".format(evaluator.evaluate(dfnn5)))
print("NN Model with 1 hidden layer and 30 neurons: AUC = {}".format(evaluatorauc.evaluate(dfnn5)))
print("____________________________________________________________________________")
print("NN Model with 3 hidden layers and 30 neurons each: Accuracy = {}".format(evaluator.evaluate(dfnn6)))
print("NN Model with 3 hidden layers and 30 neurons each: AUC = {}".format(evaluatorauc.evaluate(dfnn6)))

NN Model with 2 hidden layers and 30 neurons each: Accuracy = 0.9180110203810662
NN Model with 2 hidden layers and 30 neurons each: AUC = 0.8983686086655971
____________________________________________________________________________
NN Model with 2 hidden layers and 10 neurons each: Accuracy = 0.9167496514638518
NN Model with 2 hidden layers and 10 neurons each: AUC = 0.8957788651615598
____________________________________________________________________________
NN Model with 2 hidden layers and 20 neurons each: Accuracy = 0.9187412865962955
NN Model with 2 hidden layers and 20 neurons each: AUC = 0.896072065452911
____________________________________________________________________________
NN Model with 1 hidden layer and 30 neurons: Accuracy = 0.9112394609307575
NN Model with 1 hidden layer and 30 neurons: AUC = 0.8963076501707495
____________________________________________________________________________
NN Model with 3 hidden layers and 30 neurons each: Accuracy = 0.9223262298346

In [ ]:
# Best performing neural network model
dfnntest = mlp6_model.transform(testing_df)
print("Neural Network Model with 3 hidden layers and 30 neurons each: Accuracy = {}".format(evaluator.evaluate(dfnntest)))
print("Neural Network Model with 3 hidden layers and 30 neurons each: AUC = {}".format(evaluatorauc.evaluate(dfnntest)))

Neural Network Model with 3 hidden layers and 30 neurons each: Accuracy = 0.9188694591297275
Neural Network Model with 3 hidden layers and 30 neurons each: AUC = 0.8915997296748777


In [ ]:
nn_true = dfnntest.select(['label']).collect()
nn_pred = dfnntest.select(['prediction']).collect()
print(classification_report(nn_true, nn_pred))

              precision    recall  f1-score   support

           0       0.93      0.99      0.96      4535
           1       0.38      0.07      0.11       383

    accuracy                           0.92      4918
   macro avg       0.65      0.53      0.53      4918
weighted avg       0.88      0.92      0.89      4918

